In [34]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from pathlib import Path
# import vlc
from datetime import datetime
import random
import requests


In [9]:
def select_dropdown_item(driver, xpath, item_num, sleep_time=1.5):
    sleep(sleep_time)
    dropdown = driver.find_element(by='xpath', value=xpath)
    dropdown.click()

    sleep(sleep_time)
    item = driver.find_element(by='xpath', value=xpath.replace('label', 'panel') + f'/div/ul/li[{item_num}]')
    item.click()

def click(driver, xpath, sleep_time=1.5):
    sleep(sleep_time)
    button = driver.find_element(by='xpath', value=xpath)
    button.click()

def test_page(http_address):
    r = requests.get(http_address)
    return r.status_code == 200


In [10]:
def search_consulado(person):
    now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

    # test if system is "ON"
    if not test_page('https://agendamentosonline.mne.gov.pt/AgendamentosOnline/app/scheduleAppointmentForm.jsf'):
        print(f"{now}::{person['name']} System not responding")
        return False

    try:
        chrome_options = webdriver.ChromeOptions()

        chrome_options.add_argument(' —-headless')
        chrome_options.add_argument(' —-no-sandbox')
        chrome_options.add_argument(' —-disable-dev-shm-usage')
        driver = webdriver.Chrome('chromedriver', options=chrome_options)

        # sleep(2)
        driver.get("https://agendamentosonline.mne.gov.pt/AgendamentosOnline/app/scheduleAppointmentForm.jsf")

        # sleep(2)
        input_id = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:ccnum']")
        input_id.send_keys(person['id'])

        # sleep(1)
        birth_date = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:dataNascimento_input']")
        birth_date.send_keys(person['birth_date'])

        # sleep(1)
        pesquisar = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:searchIcon']/span")
        pesquisar.click()

        # seleciona o posto
        select_dropdown_item(driver, xpath="//*[@id='scheduleForm:postcons_label']", item_num=person['posto'])

        # aceita cookies
        click(driver, "//*[@id='j_idt316']/span")

        # seleciona a categoria
        select_dropdown_item(driver, xpath="//*[@id='scheduleForm:categato_label']", item_num=person['categoria'])

        # seleciona o ato consular
        select_dropdown_item(driver, xpath="//*[@id='scheduleForm:atocons_label']", item_num=person['ato'])

        # adicionar ato consular
        click(driver, "//*[@id='scheduleForm:bAddAto']/span")

        # concordar com condições
        click (driver, "//*[@id='scheduleForm:dataTableListaAtos:0:selCond']/div[2]")

        # calendarizar
        click(driver, "//*[@id='scheduleForm:dataTableListaAtos:0:bCal']/span")

    except Exception as e:
        print(f"{now}::{person['name']} ERROR")
        print(e)
        return False

    sleep(1)
    # pegar resultado
    try:
        resultado = driver.find_element(by='xpath', value="//*[@id='scheduleForm:j_idt171']/div[2]/table/tbody/tr[1]/td")

        if resultado.text != 'De momento não existem vagas disponíveis, por favor tente mais tarde.':
            print(resultado.text)
            print(f"{now}::{person['name']}Encontrou vagas!!!!")
            return True
        else:
            print(f"{now}::{person['name']} Não há vagas!!!!")
            return False
    except Exception as e:
        print(f"{now}::{person['name']}Não encontrou o elemento - Pode haver vagas")
        return True


In [11]:
def play_forever(mp3_path):
    i = 0
    while i<3:
        i = i+1
        print('tocando: ' + str(i))
        player = vlc.MediaPlayer(mp3_path)
        player.play()
        
        sleep(1)

        sleep(player.get_length()/1000)



In [12]:
mauricio = {
    'name': 'MAURICIO',
    'id': '9116347225', 
    'birth_date': '30-08-1977',
    'posto': 2,
    'categoria': 4,
    'ato': 4
}

selene = {
    'name': 'SELENE',
    'id': '9115088447', 
    'birth_date': '10-03-1944',
    'posto': 2,
    'categoria': 3,
    'ato': 2
}

In [6]:
mp3 = Path(r"D:\OneDrive - Agência Nacional de Águas\Fernanda\SlowPortuguese\transitions\bossa-brazil-bossa-nova-2966.mp3")

while True:
    # procura vaga maurício
    if search_consulado(selene):
        play_forever(mp3)

    if search_consulado(mauricio):
        play_forever(mp3)
        
    sleep(random.randrange(30, 180))

04-11-2022 16:13:04::SELENE ERROR
Message: session not created: This version of ChromeDriver only supports Chrome version 104
Current browser version is 106.0.5249.119 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00B878B3+2193587]
	Ordinal0 [0x00B20681+1771137]
	Ordinal0 [0x00A341A8+803240]
	Ordinal0 [0x00A55CC3+941251]
	Ordinal0 [0x00A513EF+922607]
	Ordinal0 [0x00A4EBC1+912321]
	Ordinal0 [0x00A823B0+1123248]
	Ordinal0 [0x00A8200A+1122314]
	Ordinal0 [0x00A7D5C6+1103302]
	Ordinal0 [0x00A577E0+948192]
	Ordinal0 [0x00A586E6+952038]
	GetHandleVerifier [0x00E30CB2+2738370]
	GetHandleVerifier [0x00E221B8+2678216]
	GetHandleVerifier [0x00C117AA+512954]
	GetHandleVerifier [0x00C10856+509030]
	Ordinal0 [0x00B2743B+1799227]
	Ordinal0 [0x00B2BB68+1817448]
	Ordinal0 [0x00B2BC55+1817685]
	Ordinal0 [0x00B35230+1856048]
	BaseThreadInitThunk [0x76A8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77597BBE+286]
	RtlGetAppContainerNamedOb

In [13]:
person = mauricio

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument(' —-headless')
chrome_options.add_argument(' —-no-sandbox')
chrome_options.add_argument(' —-disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# sleep(2)
driver.get("https://agendamentosonline.mne.gov.pt/AgendamentosOnline/app/scheduleAppointmentForm.jsf")

# sleep(2)
input_id = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:ccnum']")
input_id.send_keys(person['id'])

# sleep(1)
birth_date = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:dataNascimento_input']")
birth_date.send_keys(person['birth_date'])

# sleep(1)
pesquisar = driver.find_element(by='xpath', value="//*[@id='scheduleForm:tabViewId:searchIcon']/span")
pesquisar.click()


TypeError: WebDriver.__init__() got multiple values for argument 'options'

In [15]:
driver = webdriver.Chrome(options=chrome_options)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/chromium is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55c6c27af8a3 <unknown>
#1 0x55c6c249f927 <unknown>
#2 0x55c6c24d288d <unknown>
#3 0x55c6c24ce1e2 <unknown>
#4 0x55c6c2515f80 <unknown>
#5 0x55c6c25081d3 <unknown>
#6 0x55c6c24d963d <unknown>
#7 0x55c6c24da3f2 <unknown>
#8 0x55c6c2781816 <unknown>
#9 0x55c6c2784bb9 <unknown>
#10 0x55c6c278467f <unknown>
#11 0x55c6c2785035 <unknown>
#12 0x55c6c2772fbf <unknown>
#13 0x55c6c27853d0 <unknown>
#14 0x55c6c275c4d6 <unknown>
#15 0x55c6c27a07a5 <unknown>
#16 0x55c6c27a09c8 <unknown>
#17 0x55c6c27aee2f <unknown>
#18 0x7f7c5ef11134 <unknown>


In [14]:

click(driver, "//*[@id='j_idt316']/span")
# seleciona o posto
select_dropdown_item(driver, xpath="//*[@id='scheduleForm:postcons_label']", item_num=person['posto'])


In [15]:

# seleciona a categoria
select_dropdown_item(driver, xpath="//*[@id='scheduleForm:categato_label']", item_num=person['categoria'])


In [16]:

# seleciona o ato consular
select_dropdown_item(driver, xpath="//*[@id='scheduleForm:atocons_label']", item_num=person['ato'])

# adicionar ato consular
click(driver, "//*[@id='scheduleForm:bAddAto']/span")

# concordar com condições
click (driver, "//*[@id='scheduleForm:dataTableListaAtos:0:selCond']/div[2]")

# calendarizar
click(driver, "//*[@id='scheduleForm:dataTableListaAtos:0:bCal']/span")



In [17]:
 driver.close()